In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from preprocessing import preprocessing
from features_engineering import augment
from ml_methods import *
from proj1_helpers import *
import pickle as pkl

## Load the training data into feature matrix, class labels, and event ids:

In [ ]:
DATA_TRAIN_PATH = '../data/train.csv' 
DATA_TEST_PATH = '../data/test.csv'
y, tX_train, ids = load_csv_data(DATA_TRAIN_PATH)
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
(xs_train, masks_train) = preprocessing(tX_train)
(xs_test, masks_test) = preprocessing(tX_test)
y=y[:,np.newaxis]

In [ ]:
reg_f=lambda a,b,c,d,e,f:least_squares(a,b)
predictions(y, xs_train, xs_test, masks_train, masks_test, reg_f,tX_test.shape[0],2, 0, 0, 0)

In [ ]:
max_iters = 100
gamma = 10**-6
lambda_ = 10000000
reg_f=lambda a,b,c,d,e,f: reg_logistic_regression_GD(a,b,c,d,e,f)
predictions(y, xs_train, xs_test, masks_train, masks_test, reg_f,tX_test.shape[0], 2, lambda_, max_iters, gamma)

## Generate predictions and save ouput in csv format for submission:

In [ ]:
OUTPUT_PATH = '../predictions.csv' # TODO: fill in desired name of output file for submission
# y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_submission, OUTPUT_PATH)